In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Path where the multiGed datasets are stored**

In [16]:
train_file = '/content/drive/My Drive/ENLP_Project/My_try/en_fce_train.tsv'
dev_file = '/content/drive/My Drive/ENLP_Project/My_try/en_fce_dev.tsv'
test_file = '/content/drive/My Drive/ENLP_Project/My_try/en_fce_dev.tsv'

In [ ]:
!pip install happytransformer

from happytransformer import HappyTextToText


In [18]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=100)


**Remember to enter the path of the folder in which your model is saved**

In [19]:
model_path = "/content/drive/My Drive/ENLP_Project/My_try/saved_model/model/"
happy_tt = HappyTextToText("T5", "t5-base",load_path=model_path)

In [20]:
example = "grammar: This sentences, has bads grammer and spelling!"
res = happy_tt.generate_text(example, args=beam_settings)
print(res.text)

This sentence has bad grammer and spelling!


In [8]:
example_2 = "grammar: I enjoys, writtings articles ons AI."

result_2 = happy_tt.generate_text(example_2, args=beam_settings)
print(result_2.text)

I enjoy writing articles on AI.


In [ ]:
with open(dev_file, 'r') as f:

  sentence_list = []
  sent = []
  i_tags = []
  c_tags = []
  all_tags = []
  n = 0
  correct_preds = 0
  total_preds = 0
  tp = 0
  fp = 0
  fn = 0
  for i,l in enumerate(f):
    if l.split() != []:
      if l.split()[0] != ".":
        sent.append(l.split()[0])

        all_tags.append(l.split()[1])

        if l.split()[1] == 'i':
          # print("-->>",l.split()[1],l.split()[0])
          # print("n",n)
          i_tags.append(n)

        if l.split()[1] == 'c':
          c_tags.append(n) 

        n += 1

        # tags.append(l.split()[1])
        # print(l.split()[0])
      else:
        # print("entire sentence"," ".join(sent))
        sentence_list.append(" ".join(sent))
        wrong_sent = "grammar: " + " ".join(sent)
        print("---Given sentence---: ", wrong_sent)
        # print("wrong_sent",wrong_sent,wrong_sent.split() ,len(wrong_sent.split())-1)
        # if len(i_tags) > 0:
        #   print("wrong_word",sent[i_tags[0]])
        # print("----tags----",tags)
        # print("--all_tags--",all_tags)
        result_1 = happy_tt.generate_text(wrong_sent, args=beam_settings)
        # print("i_tags",i_tags)
        # print(result_1.text, result_1.text.split(), len(result_1.text.split())-1)

        print("---corrected sentence---: ",result_1.text)


        try:
          if len(i_tags) > 0:
            # print("----->>>>>>>>",i_tags)
            # print("right_word",result_1.text.split()[i_tags[0]])

            # try:
            pred_len = len(result_1.text.split())
            # if len(sent) in (pred_len,pred_len+1,pred_len-1):
            for t in i_tags:
              # print("---incorrect_word---",sent[t])
              # print("---correct_word---",result_1.text.split()[t])

      

              if sent[t] != result_1.text.split()[t]:
                tp += 1

                # correct_preds += 1

              elif sent[t] == result_1.text.split()[t]:
          #       # print("incorrect",sent[t],result_1.text.split()[t])
          #       # because the prediction is supposed to be correct, but false...spmethig like that
                fp += 1
                

              total_preds += 1
            i_tags = []


            for c in c_tags:
        #     # print("---corrections---",sent[c],result_1.text.split()[c])

              if sent[c] == result_1.text.split()[c]:
                tp += 1
              else:
        #         # print("incorrect++++++",sent[c],result_1.text.split()[c])
                fn += 1

              total_preds += 1
            c_tags = []
            # # except:
            #   print("")

          # i_tags = []
          # c_tags = []

        except:
          print("")
            

        i_tags = []
        c_tags = []

        print("\n")
        sent = []
        tags = []
        all_tags = []
        n = 0

  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  print("precision: ",precision*100)
  print("recall: ", recall*100)
  print("accuracy: ",(tp/total_preds)*100)
  print("f_0.5: ", (1.25 * precision * recall) / (0.25 * precision + recall)*100)

